<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/PVGIS_ERA5_2023_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install requests pandas

In [34]:
import pandas as pd
import requests
import io
import time

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

# PVGIS 5.3 Endpoint
base_url = "https://re.jrc.ec.europa.eu/api/v5_3/seriescalc"

for city, coords in cities.items():
    params = {
        'lat': coords['lat'],
        'lon': coords['lon'],
        'startyear': 2023,
        'endyear': 2023,
        'outputformat': 'csv',
        'raddatabase': 'PVGIS-ERA5',  # <--- FORCES PVGIS-ERA5 DATABASE
        'use_u_v': 1                   # Provides better wind vectors
    }

    try:
        print(f"Downloading ERA5 data for {city}...")
        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"❌ Server error for {city}. Code: {response.status_code}")
            continue

        # Clean the data (removing metadata and legend)
        lines = response.text.splitlines()
        header_index = next((i for i, line in enumerate(lines) if line.startswith("time,")), -1)

        if header_index == -1:
            print(f"❌ Header not found for {city}")
            continue

        # Extract 8760 hours + header
        data_lines = lines[header_index : header_index + 8761]
        df = pd.read_csv(io.StringIO("\n".join(data_lines)))

        # Format Date and Time
        df['dt_obj'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M')
        df['Date'] = df['dt_obj'].dt.date
        df['Hour'] = df['dt_obj'].dt.strftime('%H:%M')

        # Identify columns (ERA5 usually labels GHI as G(h))
        ghi_col = [c for c in df.columns if 'G(' in c][0]

        # Create clean dataset
        df_filtered = df[['Date', 'Hour', ghi_col, 'WS10m', 'T2m']].copy()
        df_filtered.columns = ['Date', 'Hour', 'GHI (W/m2)', 'Wind Speed (m/s)', 'Air Temp (C)']

        # Save individually
        filename = f"{city.lower()}_era5_2023.csv"
        df_filtered.to_csv(filename, index=False)
        print(f"✅ Saved ERA5-based file: {filename}")

        time.sleep(2) # Prevent rate limiting

    except Exception as e:
        print(f"❌ Error with {city}: {e}")

print("\nBatch process complete.")

✅ Saved ERA5-based file: turku_era5_2023.csv
✅ Saved ERA5-based file: copenhagen_era5_2023.csv
✅ Saved ERA5-based file: stockholm_era5_2023.csv
✅ Saved ERA5-based file: oslo_era5_2023.csv

Batch process complete.
